필요시 아래 라이브러리 설치

In [ ]:
pip install openai==1.10.0
pip install diffusers accelerate safetensors transformers
pip install compel

### 실사 이미지 뽑을 때 활용되는 퀄리티 관련 프롬포트 
#### positive_prompt (prompt_2) 
    가중치 부여 키워드에 사용될 단어들 
(얼굴 관련) prominent faces++, well-defined facial features++, clear expressions++, detailed faces++, emotions captured++

realism++, realistic++, true-to-life++, photo+++, best quality++, balanced++, detailed++, well-defined++, clear++, realistic color palette++, soft-focused realism++


#### negative_prompt
(얼굴 관련) no blurry faces, clear visibility of faces, no abstract faces, avoid shadows on faces

animation, anime, doll, disney, cartoons, cropped, misshapen, blurry, unfocused, desaturated, abstract, surreal, pixelated, noisy, pop art, no faces, no objects, no landscape, mutilated, disfigured, ugly, deformed

### 파일 저장 함수 

로컬에 결과 이미지를 저장하고 확인하기 위함

In [ ]:
# 파일 저장 
import pytz
import datetime

def generate_filename(model, n, m):
    korea_tz = pytz.timezone('Asia/Seoul')
    utc_now = datetime.utcnow()
    korea_now = utc_now.replace(tzinfo=pytz.utc).astimezone(korea_tz)
    korea_now = korea_now.strftime('%H:%M')
    # file_name = "/tf/notebook/SD_result_img/" + model + str(korea_now) + ".png"
    return "/tf/notebook/SD_result_img/" + model + str(n) + '_' + str(m) + str(korea_now) + ".png"

## 이미지 생성 

stable-diffusion-x1-base-1.0

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionXLPipeline
from diffusers.utils import load_image
from compel import Compel
import torch, gc
from datetime import datetime
import pytz

gc.collect()
torch.cuda.empty_cache()
print('now cache is empty')

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", 
    torch_dtype=torch.float16, 
    variant="fp16", 
    use_safetensors=True
)
pipe.to('cuda')

# Scheduler 변경 (option)
from diffusers import DDIMScheduler, DDPMScheduler, LMSDiscreteScheduler
pipe.scheduler = DDPMScheduler.from_config(pipe.scheduler.config)

prompt = "The kindergarteners are playing in the playground. Girls are wearing pink dresses, boys are wearing blue t-shirts. Children's bags are placed on the ground, and there are slides and swings in the playground."
prompt_2 = "playing+++, children+++, park+++, playground+++, swings+++, kindergarten+++, pink+++, students+++, kindergarteners+++, lawn+++, slide+++, swing+++, yellow+++"
positive_prompt = 'realistic photography+++, realism++, realistic++, super-detailed++, true-to-life++, best quality++, balanced++, well-defined++, clear++, well-defined facial features++, detailed faces++'
negative_prompt = 'CG, wallpaper, animation, anime, doll, disney, cartoons, cropped, misshapen, blurry, unfocused, desaturated, abstract, surreal, pixelated, noisy, pop art, no faces, no objects, no landscape, mutilated, disfigured, ugly, deformed, clear visibility of faces, no abstract faces, shadows on faces'

prompt_2 = prompt_2 + positive_prompt

n = 130
m = 7.5

# generator=torch.manual_seed(1)
image = pipe(prompt=prompt, prompt_2=prompt_2, negative_prompt=negative_prompt, num_inference_steps=n, guidance_scale=m).images[0]
 
file_name = generate_filename('sdxl-base-1.0_DDPM', n, m)
image.save(file_name)
image